### Fine Tuning Chatbots formating data with prompt and completion key
 - dataset: a section of the kaggle data consisting of stackoverflow python questions
 - [OpenAI-Fine-Tuning](
https://platform.openai.com/docs/guides/fine-tunin)
 - [Trouble-Shooting-Command-Line-Tool](https://community.openai.com/t/unable-to-run-the-command-line-interface-cli-for-openai/6197/12)7/12

In [9]:
#!pip install tiktoken

In [19]:
import os
import json
import openai
import tiktoken
import pandas as pd

In [2]:
openai.api_key = os.environ["OPENAI_KEY"]

In [31]:
def ask_open_ai(prompt, model="text-davinci-003"): ## "text-babbage-001"
    res = openai.Completion.create(model = model,
                                   prompt = prompt,
                                   temperature = 0,
                                   max_tokens = 256)
    return res

def token_number_count(val, encode_name):
    encoding = tiktoken.get_encoding(encode_name)
    num = len(encoding.encode(val))
    return num

In [14]:
df = pd.read_csv("./data/fine_tune_qa.csv")

In [15]:
qst, aws = df['Body'], df['Answer']

In [18]:
qa_openai_format = [ { 'prompt': q, 'completion':a  } for q,a in zip( qst, aws ) ] 
#qa_openai_format[2]

In [29]:
data_size = 500
count = 1

In [26]:
with open("./data/fine_tune_training_data.json", "w") as _f:
    _f.write("[\n")
    for entry in qa_openai_format[:data_size]:
        _f.write(json.dumps(entry))
        _f.write("\n")
        if count < data_size:
            _f.write(",")
        count += 1
    _f.write("]")

In [36]:
# example cost
token_counter = 0 # $0.0006 per 1000 tokens (training) * 4 epochs

In [37]:
for val in qa_openai_format:
    for k, v in val.items():
        token_counter += token_number_count(k, 'gpt2')
        token_counter += token_number_count(v, 'gpt2')


In [34]:
price = 0.0006*4*token_counter/1000

In [35]:
print(price)

6.526531199999999


In [40]:
# go to console and use the openai api command line tool to run the traning set
#!pip install --upgrade openai
# openai api fine_tunes.create -t "./data/fine_tune_training_data.json" -m babbage
# openai api fine_tunes.follow -i <ft-xxxxxxxxxxxxxxxx>

In [41]:
# ask_open_ai(prompt, model="text-babbage-<your-traning-id")